In [26]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = tf.keras.utils.normalize(x_train, axis=1)
x_test   = tf.keras.utils.normalize(x_test, axis=1)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=3, validation_split=.3)

Train on 42000 samples, validate on 18000 samples
Epoch 1/3
42000/42000 [==============================] - 8s 194us/step - loss: 0.3106 - acc: 0.9086 - val_loss: 0.1915 - val_acc: 0.9393
Epoch 2/3
42000/42000 [==============================] - 8s 200us/step - loss: 0.1296 - acc: 0.9603 - val_loss: 0.1366 - val_acc: 0.9567
Epoch 3/3
42000/42000 [==============================] - 8s 201us/step - loss: 0.0862 - acc: 0.9726 - val_loss: 0.1137 - val_acc: 0.9642
